In [1]:
import sympy
from model import *

sympy.init_printing()

## State transition derivation

In [3]:
dt = sympy.symbols("dt")

v_next = v + dt*a

b = sympy.Matrix([[0, -wx, -wy, -wz], [wx, 0, wz, -wy], [wy, -wz, 0, wx], [wz, wy, -wx, 0]])
qv_next = qv + (dt/2) * (b * qv)

In [7]:
qv_next.jacobian(qv), qv_next.jacobian(w)

⎛⎡        -dt⋅ωₓ    -dt⋅ω_y   -dt⋅ω_z ⎤  ⎡-dt⋅qₓ    -dt⋅q_y   -dt⋅q_z ⎤⎞
⎜⎢  1     ───────   ────────  ────────⎥  ⎢───────   ────────  ────────⎥⎟
⎜⎢           2         2         2    ⎥  ⎢   2         2         2    ⎥⎟
⎜⎢                                    ⎥  ⎢                            ⎥⎟
⎜⎢dt⋅ωₓ              dt⋅ω_z   -dt⋅ω_y ⎥  ⎢ dt⋅q_w   -dt⋅q_z    dt⋅q_y ⎥⎟
⎜⎢─────      1       ──────   ────────⎥  ⎢ ──────   ────────   ────── ⎥⎟
⎜⎢  2                  2         2    ⎥  ⎢   2         2         2    ⎥⎟
⎜⎢                                    ⎥, ⎢                            ⎥⎟
⎜⎢dt⋅ω_y  -dt⋅ω_z              dt⋅ωₓ  ⎥  ⎢ dt⋅q_z    dt⋅q_w   -dt⋅qₓ  ⎥⎟
⎜⎢──────  ────────     1       ─────  ⎥  ⎢ ──────    ──────   ─────── ⎥⎟
⎜⎢  2        2                   2    ⎥  ⎢   2         2         2    ⎥⎟
⎜⎢                                    ⎥  ⎢                            ⎥⎟
⎜⎢dt⋅ω_z   dt⋅ω_y   -dt⋅ωₓ            ⎥  ⎢-dt⋅q_y    dt⋅qₓ     dt⋅q_w ⎥⎟
⎜⎢──────   ──────   ───────      1    ⎥  ⎢──────── 

## Observation model derivation

In [16]:
# Assuming a perfect (no noise and bias) accelerometer we will
# read the acceleration of the model plus the gravity vector
# mapped to the local coordinate frame
a_exp = rot_v(a + gv, q.inverse())

In [18]:
# For the angular velocity vector, a perfect gyro (centered, no noise, no bias)
# will read the current state angular velocity
w_exp = w

In [24]:
a_exp.jacobian(a), a_exp.jacobian(qv)

⎛⎡   2     2      2      2                                                     ↪
⎜⎢q_w  + qₓ  - q_y  - q_z     2⋅q_w⋅q_z + 2⋅qₓ⋅q_y     -2⋅q_w⋅q_y + 2⋅qₓ⋅q_z   ↪
⎜⎢                                                                             ↪
⎜⎢                             2     2      2      2                           ↪
⎜⎢ -2⋅q_w⋅q_z + 2⋅qₓ⋅q_y    q_w  - qₓ  + q_y  - q_z     2⋅q_w⋅qₓ + 2⋅q_y⋅q_z   ↪
⎜⎢                                                                             ↪
⎜⎢                                                       2     2      2      2 ↪
⎝⎣  2⋅q_w⋅q_y + 2⋅qₓ⋅q_z     -2⋅q_w⋅qₓ + 2⋅q_y⋅q_z    q_w  - qₓ  - q_y  + q_z  ↪

↪ ⎤                                                                            ↪
↪ ⎥                                                                            ↪
↪ ⎥  ⎡2⋅aₓ⋅q_w + 2⋅a_y⋅q_z - 2⋅q_y⋅(a_z - g)  2⋅aₓ⋅qₓ + 2⋅a_y⋅q_y + 2⋅q_z⋅(a_z ↪
↪ ⎥  ⎢                                                                         ↪
↪ ⎥, ⎢-2⋅aₓ⋅q_z + 2⋅a_y⋅q_w